**1.Install Requiremets**

In [6]:
!pip3 install unsloth transformers datasets bitsandbytes accelerate

In [1]:
!pip install --upgrade pip
!pip install torch==2.5.1+cu118 --index-url https://download.pytorch.org/whl/cu118
!pip install torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118
Looking in indexes: https://download.pytorch.org/whl/cu118
  Using cached https://download.pytorch.org/whl/cu118/torchvision-0.21.0%2Bcu118-cp311-cp311-linux_x86_64.whl.metadata (6.1 kB)
  Using cached https://download.pytorch.org/whl/cu118/torchaudio-2.6.0%2Bcu118-cp311-cp311-linux_x86_64.whl.metadata (6.6 kB)
  Using cached https://download.pytorch.org/whl/cu118/torch-2.6.0%2Bcu118-cp311-cp311-linux_x86_64.whl.metadata (27 kB)
  Using cached https://download.pytorch.org/whl/triton-3.2.0-cp311-cp311-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (1.4 kB)
Using cached https://download.pytorch.org/whl/cu118/torchvision-0.21.0%2Bcu118-cp311-cp311-linux_x86_64.whl (6.5 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 848.7/848.7 MB ? eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.7/166.7 MB 47.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 74.8 MB/s eta 0:00:00
  Attempting uninstal

In [1]:
# Check GPU
import torch
print(torch.cuda.is_available())


True


**2.Model Loading with Unsloth:**

In [2]:
from unsloth import FastLanguageModel
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/DeepSeek-R1-Distill-Qwen-14B",
    max_seq_length = 2048,
    load_in_4bit = True,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.2.15: Fast Qwen2 patching. Transformers: 4.48.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu118. CUDA: 7.5. CUDA Toolkit: 11.8. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json:   0%|          | 0.00/182k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.35G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/236 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/6.78k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/472 [00:00<?, ?B/s]

**3.LoRA Configuration:**

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,  # Adjust the LoRA rank as needed
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],  # Specify key modules
    lora_alpha=16,
    lora_dropout=0.05,  # You can adjust dropout to prevent overfitting
    bias="none"
)

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2025.2.15 patched 48 layers with 0 QKV layers, 0 O layers and 0 MLP layers.



***Load DataSet***

In [4]:
from datasets import load_dataset
dataset = load_dataset("json", data_files="Set.json")


Generating train split: 0 examples [00:00, ? examples/s]

In [8]:
# Define the text template
prompt_template = """### Instruction:
{instruction}

### Response:
{response}"""

# Get the EOS token from the tokenizer
EOS_TOKEN = tokenizer.eos_token

# Data formatting function
def formatting_func(examples):
    texts = []
    for inst, resp in zip(examples["instruction"], examples["Response"]):
        text = prompt_template.format(instruction=inst, response=resp) + EOS_TOKEN
        texts.append(text)
    return {"text": texts}

# Apply formatting to the dataset
dataset = dataset.map(formatting_func, batched=True)


Map:   0%|          | 0/3824 [00:00<?, ? examples/s]

***Splitting the dataset***

In [9]:
dataset = dataset["train"].train_test_split(test_size=0.1)
train_dataset = dataset["train"]
eval_dataset = dataset["test"]

In [35]:
# Print the first 5 examples from the training dataset
for i in range(5):
    print("Example", i, ":\n", train_dataset[i]["text"], "\n")

Example 0 :
 ### Instruction:
Could you describe potential exploitation strategies related to CVE-2023-27555?

### Response:
Exploitation of CVE-2023-27555 could involve unauthorized individuals executing a sequence of actions to destabilize the IBM Db2 server via its ACR client affinity feature. These individuals might leverage their access to the network to transmit malformed packets or otherwise interfere with the ACR communication protocols, leading to a disruption of service. Such interference could culminate in the cessation of service to valid users or a slowing down of the system's response times. This vulnerability may serve as one component in a wider scheme to hamper company activities or might be tactically employed to soften defenses ahead of a more calculated assault.<｜end▁of▁sentence｜> 

Example 1 :
 ### Instruction:
What potential consequences exist if a cybercriminal leverages the security weakness identified by the CVE identifier CVE-2023-38268?

### Response:
If the 



---


**4. Configure Training Arguments**

In [11]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    warmup_steps=5,
    max_steps=100,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=10,
    output_dir="outputs",
    optim="adamw_8bit",
    eval_strategy="steps",  # Use eval_strategy instead of evaluation_strategy
    eval_steps=20,
)

In [12]:
from trl import SFTTrainer

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    dataset_text_field="text",
    max_seq_length=2048,
    args=training_args,
)

Converting train dataset to ChatML (num_proc=2):   0%|          | 0/3441 [00:00<?, ? examples/s]

Applying chat template to train dataset (num_proc=2):   0%|          | 0/3441 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=2):   0%|          | 0/3441 [00:00<?, ? examples/s]

Truncating train dataset (num_proc=2):   0%|          | 0/3441 [00:00<?, ? examples/s]

Converting eval dataset to ChatML (num_proc=2):   0%|          | 0/383 [00:00<?, ? examples/s]

Applying chat template to eval dataset (num_proc=2):   0%|          | 0/383 [00:00<?, ? examples/s]

Tokenizing eval dataset (num_proc=2):   0%|          | 0/383 [00:00<?, ? examples/s]

Truncating eval dataset (num_proc=2):   0%|          | 0/383 [00:00<?, ? examples/s]

**5. Start Train**

In [13]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 3,441 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 100
 "-____-"     Number of trainable parameters = 25,165,824
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: peteradel (peteradel-h) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Step,Training Loss,Validation Loss
20,2.269200,2.034655
40,1.980600,1.829682
60,1.882600,1.667268
80,1.716800,1.640834
100,1.649600,1.632152


Unsloth: Not an error, but Qwen2ForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient


TrainOutput(global_step=100, training_loss=1.9439043045043944, metrics={'train_runtime': 2148.1051, 'train_samples_per_second': 0.372, 'train_steps_per_second': 0.047, 'total_flos': 8054068769931264.0, 'train_loss': 1.9439043045043944})



---



---





**6. Model Evaluation:**

In [18]:
trainer.evaluate()

{'eval_loss': 1.6321518421173096,
 'eval_runtime': 140.8926,
 'eval_samples_per_second': 2.718,
 'eval_steps_per_second': 1.363}



---

**Save form and tokenizer:**

In [19]:
model.save_pretrained("CyberBrain_model")
tokenizer.save_pretrained("CyberBrain_model")

('CyberBrain_model/tokenizer_config.json',
 'CyberBrain_model/special_tokens_map.json',
 'CyberBrain_model/tokenizer.json')



---



---

***Upload the model to Hugging Face Hub :***

In [21]:
!pip install huggingface_hub

***Login***

In [22]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: write).
The token `Colab` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pus

***publish***

In [23]:
model.push_to_hub("PeterAdel/CyberBrain_R1")
tokenizer.push_to_hub("PeterAdel/CyberBrain_R1")


README.md:   0%|          | 0.00/628 [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/50.4M [00:00<?, ?B/s]

Saved model to https://huggingface.co/PeterAdel/CyberBrain_R1


  0%|          | 0/1 [00:00<?, ?it/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]



---



---

**Test Prompt After Train**

In [33]:
# Create a test script using the template
prompt = """### Instruction:
ما هي العواقب المحتملة لاستغلال ثغرة CVE-2023-29351؟

### Response:"""

# Convert text to token format and move input to GPU
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")

# Generate answer with specified number of new tokens
outputs_prompt = model.generate(input_ids, max_new_tokens=200)

# Decode the generated tokens and print the answer
print(tokenizer.decode(outputs_prompt[0], skip_special_tokens=True))


### Instruction:
ما هي العواقب المحتملة لاستغلال ثغرة CVE-2023-29351؟

### Response: CVE-2023-29351 يُمكن أن يستخدم لشن هجمات عبر بريد إلكتروني مزور، حيث يمكن للهاكر استغلال هذه الثغرة لخداع الضحية في إجراء تغييرات في إعدادات حسابه على ويب سكواير، مما يشمل إنشاء أو تعديل أو حذف حسابات أو تغيير إعدادات الدفع أو تغيير معلومات الاتصال.


In [34]:
# Create a test script using the template
prompt = """### Instruction:
كيف تنفذ لاستغلال ثغرة CVE-2023-29351؟

### Response:"""

# Convert text to token format and move input to GPU
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")

# Generate answer with specified number of new tokens
outputs_prompt = model.generate(input_ids, max_new_tokens=200)

# Decode the generated tokens and print the answer
print(tokenizer.decode(outputs_prompt[0], skip_special_tokens=True))


### Instruction:
كيف تنفذ لاستغلال ثغرة CVE-2023-29351؟ 

### Response: 
لتنفيذ استغلال لثغرة CVE-2023-29351، يُنصح لقراص النت بانشاء مسح للكمبيوتر المستخدم عن طريق استغلال الثغرة. يمكن للقراص النت استغلال هذه الثغرة لتنفيذ مسح محدد للكمبيوتر المستخدم، مما ينطوي على تهديدات جسيمة للنظام المستخدم.
